In [1]:
from api4jenkins import Jenkins
import time

In [2]:
j = Jenkins('http://localhost:8080/', auth=('rtj', 'rtj@jenkins'), token=True)
j.version

'2.303.3'

In [3]:
xml = """<?xml version='1.1' encoding='UTF-8'?>
... <project>
...   <builders>
...     <hudson.tasks.Shell>
...       <command>echo $JENKINS_VERSION</command>
...     </hudson.tasks.Shell>
...   </builders>
... </project>"""

In [4]:
j.create_job('freestylejob', xml)
job = j.get_job('freestylejob')
print(job)

<FreeStyleProject: http://localhost:8080/job/freestylejob/>


In [5]:
item = j.build_job('freestylejob')

In [6]:
while not item.get_build():
     time.sleep(1)
build = item.get_build()
print(build)


<FreeStyleBuild: http://localhost:8080/job/freestylejob/1/>


In [7]:
for line in build.progressive_output():
    print(line)

Started by user Shreyansh Dubey
Running as SYSTEM
Building in workspace /var/lib/jenkins/workspace/freestylejob
[freestylejob] $ /bin/sh -xe /tmp/jenkins10686892714764969861.sh
+ echo

Finished: SUCCESS



In [8]:
print(job.configure())

<?xml version="1.1" encoding="UTF-8"?><project>
  <builders>
    <hudson.tasks.Shell>
      <command>echo $JENKINS_VERSION</command>
    </hudson.tasks.Shell>
  </builders>
</project>


In [9]:
newxml = """<?xml version='1.1' encoding='UTF-8'?>
<project>
  <builders>
    <hudson.tasks.Shell>
      <command>echo this is testing!</command>
    </hudson.tasks.Shell>
  </builders>
</project>"""

In [10]:
job.configure(newxml)

<Response [200]>

In [11]:
job.description

In [12]:
job.set_description('new description')

In [13]:
job.rename('New-freestyle-Job')

<Response [302]>

## Now to reverse engineer the XML file
### Pick a manually created pipeline job, fetch the XML and create a new XML with similar configuration and Update it to a job created using code.

In [14]:
manualJob = j.get_job('Python-Application')

In [15]:
print(manualJob.configure())

<?xml version='1.1' encoding='UTF-8'?>
<flow-definition plugin="workflow-job@2.42">
  <actions>
    <org.jenkinsci.plugins.pipeline.modeldefinition.actions.DeclarativeJobAction plugin="pipeline-model-definition@1.9.3"/>
    <org.jenkinsci.plugins.pipeline.modeldefinition.actions.DeclarativeJobPropertyTrackerAction plugin="pipeline-model-definition@1.9.3">
      <jobProperties/>
      <triggers/>
      <parameters/>
      <options/>
    </org.jenkinsci.plugins.pipeline.modeldefinition.actions.DeclarativeJobPropertyTrackerAction>
  </actions>
  <description>Simple test pipeline to build python app</description>
  <keepDependencies>false</keepDependencies>
  <properties/>
  <definition class="org.jenkinsci.plugins.workflow.cps.CpsScmFlowDefinition" plugin="workflow-cps@2633.v6baeedc13805">
    <scm class="hudson.plugins.git.GitSCM" plugin="git@4.10.0">
      <configVersion>2</configVersion>
      <userRemoteConfigs>
        <hudson.plugins.git.UserRemoteConfig>
          <url>https://gith

In [30]:
customXMLForPipeline = '''<?xml version='1.1' encoding='UTF-8'?>
    <flow-definition plugin="workflow-job@2.42">
      <actions>
        <org.jenkinsci.plugins.pipeline.modeldefinition.actions.DeclarativeJobAction plugin="pipeline-model-definition@1.9.3"/>
        <org.jenkinsci.plugins.pipeline.modeldefinition.actions.DeclarativeJobPropertyTrackerAction plugin="pipeline-model-definition@1.9.3">
          <jobProperties/>
          <triggers/>
          <parameters/>
          <options/>
        </org.jenkinsci.plugins.pipeline.modeldefinition.actions.DeclarativeJobPropertyTrackerAction>
      </actions>
      <description>{}</description>
      <keepDependencies>false</keepDependencies>
      <properties/>
      <definition class="org.jenkinsci.plugins.workflow.cps.CpsScmFlowDefinition" plugin="workflow-cps@2633.v6baeedc13805">
        <scm class="hudson.plugins.git.GitSCM" plugin="git@4.10.0">
          <configVersion>2</configVersion>
          <userRemoteConfigs>
            <hudson.plugins.git.UserRemoteConfig>
              <url>{}</url>
            </hudson.plugins.git.UserRemoteConfig>
          </userRemoteConfigs>
          <branches>
            <hudson.plugins.git.BranchSpec>
              <name>*/main</name>
            </hudson.plugins.git.BranchSpec>
          </branches>
          <doGenerateSubmoduleConfigurations>false</doGenerateSubmoduleConfigurations>
          <submoduleCfg class="empty-list"/>
          <extensions/>
        </scm>
        <scriptPath>Jenkinsfile</scriptPath>
        <lightweight>true</lightweight>
      </definition>
      <triggers/>
      <disabled>false</disabled>
    </flow-definition>
'''

def createPipeline(name, xmlStr, gitRepo, desc):
  xmlStr = xmlStr.format(desc, gitRepo)
  # print(xmlStr)
  j.create_job(str(name), str(xmlStr))
  time.sleep(2)
  job = j.get_job(name)
  return job

customJob = createPipeline('customJob', customXMLForPipeline, 'https://github.com/RtjShreyD/Reverse_Shell_Python_sockets.git', 'blah blah')

# Now to Add Credentials
ref : [api4jenkins docs example](https://api4jenkins.readthedocs.io/en/latest/user/example.html#credential)
1. We need to create a new empty folder first
2. Then we need to add creds to that folder.
3. If required we can also develop on adding a adding a new job to a folder.

In [33]:
folderXml = '''<?xml version='1.0' encoding='UTF-8'?>
<com.cloudbees.hudson.plugins.folder.Folder>
 <actions/>
 <description>{}</description>
 <properties/>
 <folderViews/>
 <healthMetrics/>
</com.cloudbees.hudson.plugins.folder.Folder>'''

def createFolder(name, xmlStr, desc):
    xmlStr = xmlStr.format(desc)
    j.create_job(name, xmlStr)
    time.sleep(1)
    job = j.get_job(name)
    return job

try:
    folderJob = createFolder('Rtjtest', folderXml, 'This is a test folder created by bot')
    if folderJob:
        print(folderJob)
        print("Folder Successfully Created")
except:
    print("Error creating a folder")

<Folder: http://localhost:8080/job/Rtjtest/>
Folder Successfully Created


## Now Adding credentials to the folder created

In [41]:
credXml = '''<com.cloudbees.plugins.credentials.impl.UsernamePasswordCredentialsImpl>
  <id>002</id>
  <username>xyz</username>
  <password>abc</password>
  <description>002 for testing</description>
</com.cloudbees.plugins.credentials.impl.UsernamePasswordCredentialsImpl>'''

# folder = folderJob.Folder()

creds = folderJob.credentials.create(credXml)

In [42]:
print(creds)

None
